In [0]:
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
    
def modality(Path,index):
    X = []
    X_per_paitent = []
    p=os.listdir(Path) 
    recs_in=[]
    counter=0
    counter_2=0
    #recs_segmented=[]
    for i in p[:14]:                                                                      # Loading all the folders in the given path
        q = os.listdir(os.path.join(Path,i))     

        x = nib.load(os.path.join(Path,i,q[index]))         
        f = x.get_fdata()
        f = np.asarray(f,'float32')
        
        ct=0
        recs_in.append(f.shape[2])
        #print(counter_2)
        counter_2+=1
        for j in range(f.shape[2]):                                                        # Processing the MRI Scan in the axial view
            _slice = cv.resize(f[:,:,j],(256,256),interpolation=cv.INTER_NEAREST)             # Resizing the slice to the shape(256,256)
            if(index not in [3,4,5,6,7,8,9] and np.sum(_slice) != 0 ): 
                if index==1:
                  ct+=1 
                  counter+=1 
                 # print(counter)                                      # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001)
                else:
                                              # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001) # Standardization
            elif(index in [3,4,5,6,7,8,9]):   # if index = 3, Then it is output mask and we don't normalize or standardize it 
                _slice = np.array(_slice)
                _slice[_slice > 0] = 1.0
                _slice[_slice < 0] = 0.0
            _slice = _slice.T
            _slice = _slice[:,:,np.newaxis]
            X.append(_slice)
       
   # X=np.array(X,dtype='float32')
    return X,recs_in

In [0]:
def remove_null_samples(X_Dp, X_Flair, X_Gado, X_T1, X_T2, Y_Manual,recs): 
     
    X=[]
    Y=[]
    counter=0
    counter_2=0
    mult=0;
    count=0
    rec=[]
    keep_count=[]
    keep=[]
    print(recs)
    r=np.array(recs,dtype='float32')
    print(np.sum(r))
    print(len(X_Dp))

    for i in range(len(X_Dp)):  
        if counter==(recs[mult]):
          print(counter)
          mult+=1
          rec.append(count)
          counter=0
          print(counter_2)
          count=0
        final_slice = np.concatenate((X_Dp[i],X_Flair[i],X_Gado[i],X_T1[i],X_T2[i]), axis = -1)
        if(np.sum(final_slice) != 0):        # checking whether the final slice is empty or not             
            X.append(final_slice)
            Y.append(Y_Manual[i])
            
            count+=1
        counter+=1
        counter_2+=1

    
    rec.append(count)
#   Converting the list into array  
    X=np.array(X,dtype='float32')
    Y=np.array(Y,dtype='float32')
    rec=np.array(rec,dtype='float32')
    
    return X,Y,rec


In [0]:
def store_data(X,Y,rec):
    np.save("drive/My Drive/MS_data/X_union_new.npy",X)
    np.save("drive/My Drive/MS_data/Y_union_new.npy",Y)
    np.save("drive/My Drive/MS_data/union_new_rec_after_removal.npy",rec)



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'



# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual Segmentation
X_Dp_t,rec      =   modality(Path,0)
X_Flair_t,rec_1   =   modality(Path,1)
X_Gado_t,rec    =   modality(Path,2)
X_T1_t,rec      =   modality(Path,10)
X_T2_t,rec      =   modality(Path,11)
rec=np.array(rec_1,dtype='float32')
#np.save("drive/My Drive/MS_data/union_new_rec_before_removal.npy",rec)

In [0]:
print(rec)

[256. 256. 512. 512. 256. 512. 512. 256. 512. 256. 336. 336. 336. 336.]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'
Y_1,rec  =   modality(Path,3)
Y_2,rec  =   modality(Path,4)
Y_3,rec  =   modality(Path,5)
Y_4,rec  =   modality(Path,6)
Y_5,rec  =   modality(Path,7)
Y_6,rec  =   modality(Path,8)
Y_7,rec  =   modality(Path,9)


In [0]:
import math
def union(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7):
  Y=[]
  sum2=[]
  flag=0
  #y=np.array()
  print("A")
  for i in range (len(Y_1)):
    #print(Y_1[i])
    
        f=np.concatenate((Y_1[i],Y_2[i],Y_3[i],Y_4[i],Y_5[i],Y_6[i],Y_7[i]),axis=-1)
        sum=np.sum(f,axis=2)
       # print(sum)
        
          #print(j)
        sum_1=np.divide(sum,7)
        sum_1=np.ceil(sum_1)
        sum2.append(sum_1)

    #sum2=np.array(sum2,dtype='float32')
    
        
  return sum2

In [0]:
Y_Manual=union(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7)

A


In [0]:
import numpy as np

np.save("drive/My Drive/MS_data/Y_manual_new.npy",Y_Manual)

In [0]:
Y_manual=list(Y_Manual)

In [0]:
print(len(Y_Manual))

5184


In [0]:
Y_Manual=np.load("drive/My Drive/MS_data/Y_manual_new.npy")

In [0]:
X_Dp_t[0]

In [0]:
X, Y,rec = remove_null_samples(X_Dp_t, X_Flair_t, X_Gado_t, X_T1_t, X_T2_t, Y_manual,rec_1)

In [0]:
store_data(X,Y,rec)

In [0]:
print(rec)

[136. 143. 282. 268. 138. 280. 307. 144. 292. 141. 191. 221. 181. 216.]


In [0]:
print(np.sum(rec))

2940.0


In [0]:
print(len(X))

2940


In [0]:
import numpy as np
Y=np.load("drive/My Drive/MS_data/Y_union_new.npy")

In [0]:
Y_Manual=list(Y)

In [0]:
Y_1=Y_Manual[:1401]
Y_2=Y_Manual[1401:]

In [0]:
p=[]
y=[]
z=[]
a=[]
n=1
for i in Y_1:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  p.append(y)
  y=[]
  n+=1


#print(p)

      

In [0]:
temp=np.array(p)
np.save("drive/My Drive/MS_data/temp_1_new.npy",temp)

In [0]:
q=[]
y=[]
z=[]
a=[]
n=1401
for i in Y_2:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  q.append(y)
  y=[]
  n+=1


In [0]:
temp2=np.array(q)
np.save("drive/My Drive/MS_data/temp_2_new.npy",temp2)

In [0]:
temp=np.load("drive/My Drive/MS_data/temp_1_new.npy")
p=list(temp)

In [0]:
temp2=np.load("drive/My Drive/MS_data/temp_2_new.npy")
q=list(temp2)

In [0]:
j=1401
for i in q:
  print(j)
  p.append(i)
  j+=1

In [0]:
np.save("drive/My Drive/MS_data/Y_Manual_2_new.npy",p)

In [0]:
Y=p

In [0]:
dummy_y=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(Y[0])):
  d_temp_1=[]
  for j in range(len(Y[0][0])):
    d_temp_2=[]
    for k in range(len(Y[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_y.append(d_temp_1)
Y_dummy=np.array(dummy_y, dtype='float32')



 



In [0]:
len(Y_dummy[0][0])

1

In [0]:
X=np.load("drive/My Drive/MS_data/X_union_new.npy")

In [0]:
X=list(X)

In [0]:
dummy_x=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(X[0])):
  d_temp_1=[]
  for j in range(len(X[0][0])):
    d_temp_2=[]
    for k in range(len(X[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_x.append(d_temp_1)
X_dummy=np.array(dummy_x, dtype='float32')

In [0]:
X_dummy

In [0]:
import numpy as np

#Y=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")
X=np.load("drive/My Drive/MS_data/X_union_new.npy")
#r=np.load("drive/My Drive/MS_data/union_new_rec_after_removal.npy")

In [0]:
X=list(X)
#Y=list(Y)

In [0]:
Y=Y.tolist()

In [0]:
print(len(X))

2940


In [0]:
X_1=X[:1401]
X_2=X[1401:]

In [0]:
X_1=np.array(X_1,dtype='float32')

In [0]:
X_2=np.array(X_2,dtype='float32')

In [0]:
import numpy as np


In [0]:
X_1=np.load("drive/My Drive/MS_data/X_union_temp1.npy")

In [0]:
X_2=np.load("drive/My Drive/MS_data/X_union_temp2.npy")

In [0]:
X_2=list(X_2)

In [0]:
X_2_1=X_2[:771]
X_2_2=X_2[771:]

In [0]:
np.save("drive/My Drive/MS_data/X_union_temp2.npy",X_2)

In [0]:
np.save("drive/My Drive/MS_data/X_union_temp2_1.npy",X_2_1)
np.save("drive/My Drive/MS_data/X_union_temp2_2.npy",X_2_2)

In [0]:
print(Y[0:2])

NameError: ignored

In [0]:
np.save("drive/My Drive/MS_data/Y_union_list.npy",Y)

In [0]:
Y=np.load("drive/My Drive/MS_data/Y_union_list.npy")

In [0]:
Y=list(Y)

In [0]:
print(Y[0:2])

[array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]]), array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.]

In [0]:
X_2_1=np.load("drive/My Drive/MS_data/X_union_temp2_1_after.npy")

In [0]:
#X_2_2=np.array(X_2_2,dtype='float32')
X_2_2=X_2_2

In [0]:
print(X_2_2[0:2])

[[[[-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   ...
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]]

  [[-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   ...
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]]

  [[-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   [-0.05065425 -0.05050264 -0.03357786 -0.37272674 -0.36703324]
   ...


In [0]:
X_1=np.load("drive/My Drive/MS_data/X_union_temp1_after.npy")

In [0]:
np.save("drive/My Drive/MS_data/X_union_temp2_2_after.npy",X_2_2)

In [0]:
import numpy as np

In [0]:
X_2_2=np.load("drive/My Drive/MS_data/X_union_temp2_2_after.npy")

In [0]:
print(X_2_1)

In [0]:
X_1=list(X_1)

In [0]:
X_2_1=list(X_2_1)
X_2_2=list(X_2_2)

In [0]:
print(X_2_1[0:2])

[array([[[-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        ...,
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999]],

       [[-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        ...,
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        , -0.59300214,
         -0.55682999],
        [-0.02164003,  0.        ,  0.        ,

In [0]:
print(X_2[0:2])

In [0]:
X=X_1+X_2

In [0]:
print(X[0:2])

[array([[[ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ]],

       [[ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        , -0.03076838,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,

In [0]:
np.save("drive/My Drive/MS_data/X_union_list.npy",X)

In [0]:
extra=[0,1,6,4,6,0,5,0,4,3,1,3,3,0]

In [0]:
extra

[0, 1, 6, 4, 6, 0, 5, 0, 4, 3, 1, 3, 3, 0]

In [0]:
ext=np.array(extra,dtype='float32')

In [0]:
s=np.sum(ext)

In [0]:
print(s)

36.0


In [0]:
r=np.load("drive/My Drive/MS_data/union_new_rec_after_removal.npy")

In [0]:
rec=list(r)

In [0]:
rec

In [0]:
j=0
index=0
for i in extra:
  k=0
  
  print(i)
  index=index+rec[j]
  j+=1
  while(k<i):
    index=index+k
    print(index)
    index=int(index)
    Y.insert(index,Y_dummy)
    X.insert(index,X_dummy)

    k+=1

In [0]:
print(len(Y))

2976


In [0]:
rec=list(r)
X=list(X)
Y=list(Y)

In [0]:
X=np.array(X,dtype='float32')
Y=np.array(Y,dtype='float32')

np.save("drive/My Drive/MS_data/Actual_X.npy",X)
np.save("drive/My Drive/MS_data/Actual_Y.npy",Y)

In [0]:

Y=np.load("drive/My Drive/MS_data/Actual_X.npy")
X=np.load("drive/My Drive/MS_data/Actual_Y.npy")

In [0]:
X_1=[]
Y_1=[]


In [0]:
X=list(X)
Y=list(Y)

In [0]:
index_x_p=0
index_x=length
index_y_p=0
index_y=length
x_temp=[]
y_temp=[]
length=8



In [0]:
while index_x<=len(X) and index_y<=len(Y):
  x_temp=X[index_x_p:index_x]
  y_temp=Y[index_y_p:index_y]
  x_temp_1=np.array(x_temp,dtype='float32')
  y_temp_1=np.array(y_temp,dtype='float32')
  X_1.append(x_temp_1)
  Y_1.append(y_temp_1)
  index_y_p=index_y_p+length
  index_x_p=index_x_p+length
  index_x=index_x+length
  index_y=index_y+length


In [0]:
print(len(X_1[0]))

8


In [0]:
print(len(Y_1[0]))

8


In [0]:
np.save("drive/My Drive/MS_data/train_X.npy",X_1)
np.save("drive/My Drive/MS_data/train_Y.npy",Y_1)

In [0]:
Y_1[0]

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 13 18:56:04 2019

@author: Krishna Chandra
"""
import tensorflow as tf
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv3D, UpSampling3D
from keras.layers.pooling import MaxPooling3D
from keras.layers.merge import concatenate


def Convolution(input_tensor,filters):
    
    x = Conv3D(filters=filters,kernel_size=(3, 3, 3),padding = 'same',strides=(1, 1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x

def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)
    
    maxp_1 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_1)
    
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_2)
    
    conv_3 = Convolution(maxp_2,128)
    
    maxp_3 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_3)
    
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_4)
    
    conv_5 = Convolution(maxp_4,256)
   
    upsample_6 = UpSampling3D((2, 2, 2)) (conv_5)

    upsample_6 = concatenate([upsample_6, conv_4],axis=1)
    
    conv_6 = Convolution(upsample_6,128)
    upsample_7 = UpSampling3D((2, 2, 2)) (conv_6)
    
    upsample_7 = concatenate([upsample_7, conv_3],axis=1)
    
    conv_7 = Convolution(upsample_7,64)
    upsample_8 = UpSampling3D((2, 2, 2)) (conv_7)
    
    upsample_8 = concatenate([upsample_8, conv_2],axis=1)

    conv_8 = Convolution(upsample_8,32)
    upsample_9 = UpSampling3D((2, 2, 2)) (conv_8)
    
    upsample_9 = concatenate([upsample_9, conv_1],axis=1)
    
    conv_9 = Convolution(upsample_9,32)
    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid') (conv_9)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model



In [0]:

from keras import backend as K
import numpy as np
import tensorflow as tf

# Computing Dice_Coefficient
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

Using TensorFlow backend.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv3D, UpSampling3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

X_train , X_test, Y_train, Y_test = train_test_split(X_1, Y_1, test_size=0.15, random_state=32)


# Loding the modified U-net 
model = model(input_shape = (8,256,256,5))
model.summary()

checkpointer = ModelCheckpoint('Modified_UNet.h5', verbose=1)
callback_list=[checkpointer]

# Compiling the model
k_adam=Adam(lr=0.001)
model.compile(optimizer=k_adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])
# Fitting the model over the data
history = model.fit(X_train,Y_train,batch_size=32,epochs=60,validation_split=0.20,verbose=1,initial_epoch=0,callbacks=callback_list)

# Saving the model
model.save('Modified_UNet.h5')
history.history

# Evaluating the model on the training and testing data 
model.evaluate(x=X_train, y=Y_train, batch_size=32 , verbose=1, sample_weight=None, steps=None)
model.evaluate(x=X_test, y=Y_test, batch_size=32, verbose=1, sample_weight=None, steps=None)


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 8, 256, 256, 0                                            
__________________________________________________________________________________________________
conv3d_59 (Conv3D)              (None, 8, 256, 256,  4352        input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_55 (BatchNo (None, 8, 256, 256,  128         conv3d_59[0][0]                  
__________________________________________________________________________________________________
activation_55 (Activation)      (None, 8, 256, 256,  0           batch_normalization_55[0][0]     
____________________________________________________________________________________________

ValueError: ignored

In [0]:
print(X_train[0].shape)